In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
os.chdir('/home/jovyan/work/carlos/complete_execution_andalucia')


In [ ]:
USE_CASE = "andalusia"

BASE_PATH = f"./data/use_case_{USE_CASE}"

METADATA_PATH = os.path.join(BASE_PATH, "metadata")
PRODUCT_MAPPING_FILEPATH = os.path.join(METADATA_PATH, "Product_Mapping.csv")

RESULTS_PATH = os.path.join(BASE_PATH, "results")
REPRESENTATIVENESS_FILEPATH = os.path.join(RESULTS_PATH, "FADN_Representativeness_2014.csv")



In [ ]:
MICRODATA_PATH = os.path.join(BASE_PATH, "microdata")
# This microdata is the stack of all the microdata for a given use case, combining all the years available

microdata = pd.DataFrame()

for file in os.listdir(MICRODATA_PATH):
    microdata = pd.concat([microdata, pd.read_csv(os.path.join(MICRODATA_PATH, file))])


In [ ]:
microdata

In [ ]:
product_mapping = pd.read_csv(PRODUCT_MAPPING_FILEPATH)
representativeness = pd.read_csv(REPRESENTATIVENESS_FILEPATH)

display(product_mapping.head())
display(representativeness.head())
display(microdata.head())

In [ ]:
subsidies_dict = {
    # Decoupled payments
    'M_S_1150_FI_BU_V': "Basic payment scheme", 
    'M_S_1400_FI_BU_V': "Payment for agricultural practices beneficial for the climate and the environment",
    'M_S_1600_FI_BU_V': "Payment for young farmers ",
    'M_S_1700_FI_BU_V': "Small farmers scheme ",

    # Crops
    'M_S_23111_FI_BU_V': "Cereals",
    'M_S_23112_FI_BU_V': "Oilseeds",
    'M_S_23113_FI_BU_V': "Protein crops",
    'M_S_2313_FI_BU_V': "Potatoes",
    'M_S_2315_FI_BU_V': "Vegetables",
    'M_S_2317_FI_BU_V': "Rice",
    'M_S_2318_FI_BU_V': "Grain legumes",
    'M_S_2319_FI_BU_V': "Arable crops not defined",
    'M_S_2322_FI_BU_V': "Crop specific payment for cotton",
    'M_S_2323_FI_BU_V': "National restructuring programme for the cotton sector",

    'M_S_23312_FI_BU_V': "Nuts",
    'M_S_2333_FI_BU_V': "Citrus plantations",
    'M_S_2334_FI_BU_V': "Olive plantations",
    'M_S_2335_FI_BU_V': "Vineyards",
    'M_S_2339_FI_BU_V': "Permanent crops not mentioned elsewhere",

    # Animals
    'M_S_2341_FI_BU_V': "Dairy",
    'M_S_2342_FI_BU_V': "Beef and veal",
    'M_S_2343_FI_BU_V': "Cattle (type not specified)",
    'M_S_2344_FI_BU_V': "Sheep and goat",
    'M_S_2349_FI_BU_V': "Animals not mentinoed elsewhere",
    'M_S_2490_FI_BU_V': "Other coupled payments not mentioned elsewhere",

    # Grants and subsidies of exceptional character
    'M_S_2890_FI_BU_V': "Other grants and subsidies of exceptional character",
    'M_S_2900_FI_BU_V': "Other direct payments not mentioned elsewhere",

    # Rural development
    'M_S_3100_FI_BU_V': "Investment subsideis for agriculture",
    'M_S_3300_FI_BU_V': "Agri-environmen-climate and aniumal welfare payments",
    'M_S_3350_FI_BU_V': "Organic farming",
    'M_S_3500_FI_BU_V': "Payments to areas facing natural or other specific constraints",
    'M_S_3610_FI_BU_V': "Investment in forest area development and climate services and forest conservation support",
    'M_S_3620_FI_BU_V': "Natura 2000 payments for forestry and fores-evironmental and climate services and forest conservatino support",
    'M_S_3750_FI_BU_V': "Support to restoring agricultural production potential damaged by natural distaters and catastrophic events and introduction of appropriate prevention actions",
    'M_S_3900_FI_BU_V': "Other payments for rural development",

    # Grants and subsidies on costs
    'M_S_4200_FI_BU_V': "Motor fuels",
    'M_S_9000_FI_BU_V': "Differences from the previous accounting years",
    }


# subsidy code: [crop/animal code linked]
subsidies_crop_link = {
    # Crops
    # "Cereals"
    23111: [
        10110, # Common wheat and spelt
        10120, # Durum wheat
        10130, # Rye ad winter cereal mixtures
        10140, # Barley
        10150, # Oats and spring cereal mixtures (mixed grain aother than maslin)
        10160, # Grain maize and corn-cob mix
        10190, # Triticale, sorghum, and other cereals
        ],
    # "Oilseeds"
    23112: [
        10604, # Rapeseed, 
        10606, # Soybean, 
        10607, # Linseed (oil flax)
        10609, # Fibre flax
        10608, # Other crops grown for their oil content, harvested as dry grains, which are not mentioned elsewhere. Includes mustard, poppy, safflower (carthamus), sesame seed, earth almond, peanuts, pumpkins for oil, flax other than fibre flax if not recorded under ► Category 10607.
            ], 
    # "Protein crops"
    23113: [
        10210, # Field peas, beans and sweet lupins
        10220, # Lentils, chickpeas and vetches
        10290, # Other protein crops
        ],
    # "Potatoes"
    2313: [10300, # Potatoes (including early potatoes and seed potatoes)
           10310, # Potatoes of which potatoes for starch
           10390, # Potatoes of which other potatoes
           ],
    # "Vegetables"
    2315: [
        10711, #Fresh vegetables (incl. melons) and strawberries - Open field
        10712, # Fresh vegetables (incl. melons) and strawberries – Market gardening
        10720, # Fresh vegetables(incl. melons) and strawberries – Under glass or under high accessible cover
        10731, # Cauliflower and broccoli
        10732, # Lettuce
        10733, # Tomatoes
        10734, # Sweet corn
        10735, # Onions
        10736, # Garlic
        10737, # Carrots
        10738, # Strawberries
        10739, # Melons
        10790, # Other vegetables
    ],
    # "Rice"
    2317: [
        10170, # Rice
        ],
    # "Grain legumes",
    2318: [
        10210, # Field peas, beans and sweet lupins
        10220, # Lentils, chickpeas and vetches
    ], 
    # "Arable crops not defined",
    2319: [], # Unknown
    # "Crop specific payment for cotton",
    2322: [
        10603, # Cotton
           ], 
    # "National restructuring programme for the cotton sector",
    2323: [
        10603, # Cotton
           ], 
    # Nuts
    23312: [40130, #"Nuts",
            ],
    # "Citrus plantations",
    2333: [
        40200, # Citrus fruits. Oranges, small citrus fruits, lemons, limes, pomelos, grapefruits and other citrus fruits
        40210, # - of which orangesOranges Excludes bitter oranges (to be recorded under ► Category 40200)
        40230, # - of which lemonsLemons
    ], 
    # "Olive plantations",
    2334: [
        40310, # Table olives
        40320, # Olives for oil production
        40330, # Olive oil
        40340, # Olive by-products
    ], 
    # "Vineyards",
    2335: [
        40411, # Wine with protected designation of origin (PDO)
        40412, # Wine with protected geographical indication (PGI)
        40420, # Other wines
        40430, # Grapes for table use
        40440, # Grapes for raisins
        40451, # Grapes for wine with protected designation of origin (PDO)
        40452, # Grapes for wine with protected geographical indication (PGI)
        40460, # Grapes for other wines
        40470, # Miscellaneous products of vines: grape must, juice,  brady, vinegar and others produced on the farm
    ], 
    # "Permanent crops not mentioned elsewhere",
    2339: [], 

    # Animals
    # "Dairy"
    2341: [
        261, # Dairy cows
        262, # Buffalo dairy cows
    ],
    # "Beef and veal"
    2342: [
        210, # Bovine animals, less than one year old, male and female
        220, # Male bovine animals, 1 to less than 2 years old
        230, # Heifers, 1 to less than 2 years old
        240, # Male bovine animals, 2 years old and over
    ],
    # "Cattle (type not specified)"
    2343: [
        210, # Bovine animals, less than one year old, male and female
        220, # Male bovine animals, 1 to less than 2 years old
        230, # Heifers, 1 to less than 2 years old
        240, # Male bovine animals, 2 years old and over
    ],
    # "Sheep and goat"
    2344: [
        "K_PR_311_Q", # Sheep's production
        "K_PR_321_Q", # Goat's production
    ],
    # "Animals not mentinoed elsewhere"
    2349: [],
    # "Other coupled payments not mentioned elsewhere"
    2490: [],
}


animal_codes_subsidides = [2341, 2342, 2343, 2344, 2349, ]
crops_codes_subsidides  = [23111, 23112, 23113, 2313, 2315, 2317, 2318, 2319, 2322, 2323, 23312, 2333, 2334, 2335, 2339, ]


decoupled_subsidies = [
    "M_S_1150_FI_BU_V", 
    "M_S_1400_FI_BU_V", 
    "M_S_1600_FI_BU_V", 
    "M_S_1700_FI_BU_V", ]


In [ ]:
subsidies_df = pd.DataFrame(subsidies_dict, index=[0]).transpose()

subsidies_df["Variable"] = subsidies_df.index

subsidies_df = subsidies_df.reset_index(drop=True).rename(columns={0: "Description"})[["Variable", "Description"]]

# Add Code column
subsidies_df["Subsidy_Code"] = subsidies_df.apply(lambda x: int(x["Variable"].replace("M_S_", "").replace("_FI_BU_V", "")), axis=1)

# Add coupled column
subsidies_df["Coupled"] = subsidies_df.apply(lambda x: "N" if x["Variable"] in decoupled_subsidies else "Y", axis=1)

# Aggregated_product
subsidies_df["Aggregated_product"] = ""

# Economic_compensation
subsidies_df["Economic_compensation"] = ""

# StartYear
subsidies_df["StartYear"] = ""

# EndYear
subsidies_df["EndYear"] = ""

code_labels = {
    1150: "Basic", 
    1400: "Greening", 
}

# Label
subsidies_df["Label"] = subsidies_df.apply(lambda x: code_labels[x["Subsidy_Code"]] if x["Subsidy_Code"] in code_labels.keys() else "", axis=1)

display(subsidies_df)


### Decoupled subsidies

In [ ]:
for idx in subsidies_df.index:
    
    if subsidies_df.at[idx, "Coupled"] == "N":
        subsidy_var = subsidies_df.at[idx, "Variable"]
        avg_value = microdata[microdata[subsidy_var]>0][subsidy_var].mean()
        print(subsidy_var, avg_value)

        subsidies_df.at[idx, "Economic_compensation"] = avg_value

display(subsidies_df[subsidies_df["Coupled"]=="N"])


In [ ]:
microdata2 = pd.DataFrame()

microdata2["YEAR"] = microdata["YEAR"] 

## Compute perception by hectare
for code in subsidies_crop_link:
    
    # Subsidy variable
    subsidy_var = f"M_S_{code}_FI_BU_V"    
    microdata2[subsidy_var] = microdata[subsidy_var]
    
    print(code)
    
    # Check available total area variables linked to subsidy
    ta_variables = [f"I_A_{crop}_TA" for crop in subsidies_crop_link[code] if f"I_A_{crop}_TA" in microdata.columns]

    # Copy area data to microdata2 if exists
    for ta_var in ta_variables:
        microdata2[ta_var] = microdata[ta_var]

    microdata2[f"{code}-total-area"] = microdata2.apply(lambda x: x[ta_variables].sum(), axis=1)

    print("AREA", microdata2[f"{code}-total-area"].sum())
    
    #microdata2[f"{code}-total-area"] = microdata2[ta_variables].sum()

    microdata2[f"{code}-perception-ha"] = microdata2.apply(lambda x: x[subsidy_var]/ x[f"{code}-total-area"] if x[f"{code}-total-area"]>0 else 0, axis=1)
    
    print("PERC", microdata2[microdata2[f"{code}-perception-ha"]>0][f"{code}-perception-ha"].mean())
    print(ta_variables)

    print(".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")

In [ ]:
for c in [cc for cc in microdata2.columns if cc.endswith("perception-ha")]:
    print(microdata2[microdata2[c]>0][c].mean())

### Assign start year and end year

In [ ]:
subsidies_value_variables = [c for c in microdata.columns if c.startswith("M") and c.endswith("V")]

# Subsidies with at least one record in Andalusian microdata
subsidies_codes = sorted([c for c in subsidies_value_variables if microdata[c].dropna().shape[0]>0])

microdata["YEAR"] = microdata["YEAR"].astype(int)

for idx in subsidies_df.index:
    variable = subsidies_df.at[idx, "Variable"]

    selection = microdata[["YEAR", variable]]
    
    min_year = selection[selection[variable]>0]["YEAR"].min()
    max_year = selection[selection[variable]>0]["YEAR"].max()

    subsidies_df.at[idx, "StartYear"] = min_year
    subsidies_df.at[idx, "EndYear"] = max_year

In [ ]:
subsidies_df

## Assign perception

In [ ]:
tot = representativeness["total_area"].sum()
representativeness["%UAA"] = representativeness.apply(lambda x: x["total_area"]/tot, axis=1)

In [ ]:
res = subsidies_df[subsidies_df["Coupled"]=="N"].copy(deep=True)

for idx in subsidies_df.index:

    subsidy_code = subsidies_df.at[idx, "Subsidy_Code"]

    print(subsidy_code)
    if subsidy_code in subsidies_crop_link.keys():
        
        # Get FADN products associated with this subsidy
        FADN_products = subsidies_crop_link[subsidy_code]
        print(f"    {FADN_products}")

        # Get representativeness of such products | Combine conventional and organic
        FADN_products_representativeness_all = representativeness[representativeness["fadn_code"].apply(lambda x: x in FADN_products)][["fadn_code", "%UAA", "product_group"]]#.groupby("fadn_code").sum()

        for agg in FADN_products_representativeness_all["product_group"].unique():
            selection_agg = FADN_products_representativeness_all[FADN_products_representativeness_all["product_group"]==agg]

            # Compute economic compensation for the given aggregation
            # Compute total representativeness
            total_rep = selection_agg["%UAA"].sum()

            # Normalise representativeness
            selection_agg["%UAA norm"] = selection_agg.apply(lambda x: x["%UAA"]/total_rep, axis=1)

            # Get perceptions by FADN code
            FADN_perception = microdata2[f"{subsidy_code}-perception-ha"]
            FADN_perception = FADN_perception[FADN_perception>0].mean()

            if FADN_perception > 0:    
                
                # Add weighted perception
                selection_agg["weighted perception"] = selection_agg.apply(lambda x: x["%UAA norm"]*FADN_perception, axis=1)
                
                weighted_perception = selection_agg["weighted perception"].sum()

            else:
                weighted_perception = 0

            row = pd.DataFrame({
                'Variable': [subsidies_df.at[idx, "Variable"]], 
                'Description': [subsidies_df.at[idx, "Description"]], 
                'Subsidy_Code': [subsidies_df.at[idx, "Subsidy_Code"]], 
                'Coupled': [subsidies_df.at[idx, "Coupled"]],
                'Aggregated_product': [agg], 
                'Economic_compensation': [weighted_perception], 
                'StartYear': [subsidies_df.at[idx, "StartYear"]], 
                'EndYear': [subsidies_df.at[idx, "EndYear"]],
                'Label': [subsidies_df.at[idx, "Label"]]})
            
            res = pd.concat([res, row], axis=0)


In [ ]:
res = res[["Description", "Subsidy_Code", "Coupled", "Aggregated_product", "Economic_compensation", "StartYear", "EndYear", "Label"]].reset_index(drop=True)

In [ ]:
res["Economic_compensation"] = res.apply(lambda x: 100*x["Economic_compensation"] if x["Coupled"]=="Y" else x["Economic_compensation"], axis=1)


In [ ]:
res = res.fillna(0)

display(res)

In [ ]:
org_policies = pd.DataFrame(
    [["Organic conversion of crops",9900,"Y","ORG_FRUIT",311.245,2015,2020, "",], 
    ["Organic conversion of crops",9900,"Y","ORG_CITRUS",311.245,2015,2020, "",], 
    ["Organic conversion of crops",9900,"Y","ORG_GRAZ",180.73,2015,2020, "",], 
    ["Organic olive conversion",9901,"Y","ORG_OLIVE",272.69,2015,2020, "", ],], columns=res.columns)


res = pd.concat([res, org_policies], axis=0).reset_index(drop=True)


In [ ]:
res = res[res["Economic_compensation"]>0].reset_index(drop=True)

In [ ]:
display(res)

In [ ]:
res.to_csv("./data/use_case_andalusia/metadata/subsidies.csv", index=False)